In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import pymorphy2
import re
from numba import jit

## Описание полей
 * __id__ &mdash; внутренний идетификатор
 * __name__ &mdash; название вакансии
 * __description__ &mdash; текст вакансии
 * __target__ &mdash; класс заинтересованности

In [ ]:
posConv={'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB', 'PRTF': '_PRT', 'GRND': '_GRND', 'NUMR': '_NUMR'}

def getVacancyVector(name, description, needPos=False):
    text = name + ' ' + description
    text = BeautifulSoup(text, 'lxml').get_text()
    
    words=[a[0] for a in re.findall("([a-zA-ZА-ЯЁа-яё]+(-[a-zA-ZА-ЯЁа-яё]+)*)", text)]
    reswords=[]

    for w in words:
        if(len(w)>2):
            wordform=morph.parse(w)[0]
            if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB', 'PRTF', 'GRND', 'NUMR']:
                if needPos!=None:
                    reswords.append(wordform.normal_form+posConv[wordform.tag.POS])
                else:
                    reswords.append(wordform.normal_form)
            
    return " ".join(reswords)

In [ ]:
morph=pymorphy2.MorphAnalyzer() 

In [ ]:
train_data = pd.read_csv('data/train_processed.csv', sep='\t', index_col='id')

In [ ]:
train_data

In [ ]:
%%time

train_data['processed'] = train_data.apply(lambda row: getVacancyVector(row['name'], row['description']), axis=1)
#train_data.name = train_data.apply(lambda row: getVacancyVector(row['name'], True), axis=1)

In [ ]:
train_data.to_csv('data/train_processed2.csv', '\t')

In [ ]:
test_data = pd.read_csv('data/test.csv', sep='\t', index_col='id')

In [ ]:
%%time

test_data.description = test_data.apply(lambda row: getVacancyVector(row['description'], True), axis=1)

In [ ]:
%%time

test_data.name = test_data.apply(lambda row: getVacancyVector(row['name'], True), axis=1)

In [ ]:
test_data.to_csv('data/test_processed.csv', '\t')

In [ ]:
other_data = pd.read_csv('data/other.txt', sep='\t')

In [ ]:
other_data.head()

In [ ]:
%%time

other_data.description = other_data.apply(lambda row: getVacancyVector(row['description'], True), axis=1)
other_data.name = other_data.apply(lambda row: getVacancyVector(row['name'], True), axis=1)

In [ ]:
other_data.to_csv('data/oter_processed.csv', '\t')

In [ ]:
test_data = pd.read_csv('data/test.csv', sep='\t', index_col='id')

In [ ]:
%%time

test_data.description = test_data.apply(lambda row: getVacancyVector(row['description'], True), axis=1)

In [ ]:
%%time

test_data.name = test_data.apply(lambda row: getVacancyVector(row['name'], True), axis=1)

In [ ]:
test_data.to_csv('data/test_processed.csv', '\t')